In [1]:
#!pip install tradingeconomics
#!pip install config

In [2]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the config file
config.read('config.ini')


['config.ini']

In [3]:
#create a config file with your API key

import tradingeconomics as te
# get api_key from config file
api_key = config.get('irishConstructionData', 'api_key')
te.login(api_key)
data = te.getHistoricalData(country='Ireland', indicator='Construction Output', initDate='2015-01-01')
print(data)



HTTP Error 403: No Access to this country as free user.
None
